In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
import lightgbm as lgb
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm, tqdm_notebook
import pickle
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import math
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import json
import gc
import re
warnings.filterwarnings('ignore')

In [3]:
import os
os.chdir('/cos_person/ChinaCom/CCF/WorkPiece/data')
train = pd.read_csv('../data/first_round_training_data.csv')
test = pd.read_csv('../data/first_round_testing_data.csv')

In [4]:
len(train),len(test)

(6000, 6000)

In [7]:
features = [c for c in test.columns if c!='Group']
cat_feats = ['Attribute4',
'Attribute5',
'Attribute6',
'Attribute7',
'Attribute8',
'Attribute9',
'Attribute10',
'Parameter5',
'Parameter6',
'Parameter7',
'Parameter8',
'Parameter9',
'Parameter10']

use_cate = [c for c in cat_feats if 'Para' in c]
col_only_train = [c for c in test.columns if c != 'Group']
all_feat = [c for c in train.columns if c != 'Quality_label']
data = pd.concat([train,test])

def nnq_encode(data,en_col,use_col):
    data[en_col + '_nnq_of_' + use_col] = data[en_col].map( data.groupby([en_col])[use_col].nunique() )
    features.append( en_col + '_nnq_of_' + use_col )
    return data

for en_col in use_cate:
    for use_col in cat_feats:
        if en_col != use_col:
            data = nnq_encode(data,en_col,use_col)
for en_col in use_cate:
    for use_col in use_cate:
        if en_col != use_col:
            colname =  en_col +'_count_' + use_col
            features.append( colname)
            data[ colname] = data[en_col].astype(str) + "|" + data[use_col].astype(str)
            data[ colname] = data.groupby([ colname ])[colname].transform('count')
for en_col in use_cate:
    for use_col in all_feat:
        if en_col != use_col:
            colname =  en_col +'_mean_' + use_col
            features.append( colname)
            data[ colname] = data[en_col].map( data.groupby([en_col])[use_col].mean() )
            colname =  en_col +'_std_' + use_col
            features.append( colname)
            data[ colname] = data[en_col].map( data.groupby([en_col])[use_col].std() )


In [8]:
classMap = {'Excellent':0,
'Good':1,
'Pass':2,
'Fail':3,}
tr_index = ~data.Quality_label.isnull()
train_df = data[tr_index][features +['Quality_label']].reset_index(drop=True)
train_df['Quality_label'] = train_df['Quality_label'].map(classMap)

test_df = data[~tr_index].reset_index(drop=True)
id_test = test_df.Group.values


X_train = train_df[features]    
y = train_df.Quality_label
X_test = test_df[features]
del train,test
del train_df,test_df

In [11]:
# train_df

In [ ]:
lgb_paras = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'learning_rate': 0.05,
        'num_leaves': 32,
        # 'lambda_l1': 0.01,
        # 'lambda_l2': 10,
        'num_class': 4,
        'max_depth': -1,
        'seed': 42,
        'feature_fraction': 0.8,
        #         'bagging_fraction': 0.8,
        #         'bagging_freq': 4,
        'verbose': 1
    }
from sklearn.metrics import f1_score, confusion_matrix
all_preads = []
skf = StratifiedKFold(n_splits=5, random_state=2029, shuffle=True)
def f1_weighted(preds, train_data):
    y_true = train_data.label
    preds = np.argmax(preds.reshape(4, -1), axis=0)
    score = f1_score(y_true, preds, average='weighted')
    return 'f1_weighted', score, True
evals_result = {}
for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
    train_x, test_x, train_y, test_y = X_train[features].iloc[train_index], X_train[features].iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    dtrain = lgb.Dataset(train_x, train_y)

    dvalid = lgb.Dataset(test_x, test_y)
#     clf = lgb.train(
#         params=lgb_paras,
#         train_set=dtrain,
#         num_boost_round=10000,
#         valid_sets=[dtrain, dvalid],
#         evals_result=evals_result,
#         early_stopping_rounds=100,
#         valid_names=["train", "valid"],
#         # categorical_feature=cate_cols,
#         verbose_eval=100)
    
    lgb_modelall = lgb.train(lgb_paras, dtrain,
                             valid_sets=[dtrain, dvalid],
                             num_boost_round=1000,
                             early_stopping_rounds=100,
                             valid_names=["train", "valid"],
                             evals_result=evals_result,
                             verbose_eval=50,
                             feval=f1_weighted)
    pred = lgb_modelall.predict(X_test)
    all_preads.append( pred )


    

Training until validation scores don't improve for 100 rounds.
[50]	train's multi_logloss: 0.884214	train's f1_weighted: 0.687493	valid's multi_logloss: 1.09958	valid's f1_weighted: 0.51745
[100]	train's multi_logloss: 0.723673	train's f1_weighted: 0.76786	valid's multi_logloss: 1.09925	valid's f1_weighted: 0.512451
[150]	train's multi_logloss: 0.608637	train's f1_weighted: 0.838183	valid's multi_logloss: 1.11755	valid's f1_weighted: 0.505265
Early stopping, best iteration is:
[51]	train's multi_logloss: 0.880174	train's f1_weighted: 0.687505	valid's multi_logloss: 1.09908	valid's f1_weighted: 0.518638
Training until validation scores don't improve for 100 rounds.
[50]	train's multi_logloss: 0.886561	train's f1_weighted: 0.694015	valid's multi_logloss: 1.10078	valid's f1_weighted: 0.510211
[100]	train's multi_logloss: 0.725004	train's f1_weighted: 0.770889	valid's multi_logloss: 1.10016	valid's f1_weighted: 0.511115


In [26]:
mean_pread = np.mean(all_preads,axis=0)
cols = ['Excellent ratio','Good ratio','Pass ratio','Fail ratio']

sub_prob  = pd.DataFrame(mean_pread,columns=cols)
len(sub_prob),len(id_test)
sub_prob['Group'] = id_test

sub_prob['Group'] = sub_prob['Group'].map(int)
sub_prob = sub_prob.groupby([ 'Group'])[cols].median().reset_index()

sub_prob.to_csv('../submission/lgb_fisrt.csv',index=False)

In [25]:
all_preads[0][0]

array([0.19381562, 0.16818632, 0.20587343, 0.43212463])